In [ ]:
## why not put attention heads as 3rd dimension? (BATCH_SIZE, ATTENTION_HEADS, SEQ_LEN, D_MODEL)
## Apply padding masks to multi head attention
## how to stack transformer encoder decoder layers

In [29]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np

In [49]:
debug = True
def logs(s):
    if debug:
        print(s)

In [79]:
t = torch.rand(2, 1, 5, 3)

In [80]:
t

tensor([[[[0.5520, 0.8752, 0.9604],
          [0.9028, 0.3016, 0.8522],
          [0.2651, 0.7454, 0.8019],
          [0.4348, 0.0396, 0.2529],
          [0.8547, 0.4077, 0.3363]]],


        [[[0.3557, 0.4578, 0.1522],
          [0.5687, 0.0858, 0.4042],
          [0.4029, 0.7768, 0.1005],
          [0.8105, 0.0802, 0.7300],
          [0.4231, 0.9843, 0.3933]]]])

In [81]:
F.softmax(t, dim = 3)

tensor([[[[0.2573, 0.3555, 0.3871],
          [0.4002, 0.2194, 0.3804],
          [0.2311, 0.3736, 0.3953],
          [0.3989, 0.2686, 0.3325],
          [0.4474, 0.2861, 0.2664]]],


        [[[0.3421, 0.3788, 0.2791],
          [0.4056, 0.2503, 0.3441],
          [0.3132, 0.4553, 0.2315],
          [0.4159, 0.2004, 0.3837],
          [0.2686, 0.4707, 0.2607]]]])

## Building Blocks

#### Scaled Dot Product Attention

In [117]:
class ScaledDotProductAttention(nn.Module) :

    def __init__(self, 
                 n_heads:int = 8,
                 d_model:int = 512,
                 mask:bool = False
        ) -> None :

        """
        Args:
            n_heads (int): Number of heads in the multi head attention. Defualts to 8
            d_model (int, optional): Dimension of the input. Defaults to 512.
            mask (bool, optional): Whether to apply masking. Defaults to False
        """

        super(ScaledDotProductAttention, self).__init__()

        self.n_heads = n_heads
        self.d_model = d_model
        self.mask = mask
        self.d_k = int(d_model/n_heads)

    def forward(self,
                key : torch.Tensor,
                query : torch.Tensor,
                value : torch.Tensor
        ) -> torch.Tensor :

        """
        Calculate scaler dot product of key, query and values as described in https://arxiv.org/pdf/1706.03762.pdf

        Args:
            key (torch.Tensor): Key tensor. Shape = (n_heads, batch_size, seq_len, d_model/n_heads)
            query (torch.Tensor): Query tensor. Shape = (n_heads, batch_size, seq_len, d_model/n_heads)
            value (torch.Tensor): Value tensor. Shape = (n_heads, batch_size, seq_len, d_model/n_heads)

        Returns:
            value_with_attention: Value with attention applied. Shape = (n_heads, batch_size, seq_len, d_model/n_heads)
        """

        assert key.size() == query.size() == value.size(), "Key, query and value must have same shape"

        batch_size, seq_len = key.size(1), key.size(2)

        attention_scores = torch.matmul(query, key.transpose(2, 3))/torch.sqrt(torch.tensor(self.d_k))
        attention_scores = torch.softmax(attention_scores, dim = 3)
        
        if self.mask :
            mask = torch.ones(self.n_heads, batch_size, seq_len, seq_len)
            mask = torch.tril(mask)
            attention_scores = torch.matmul(attention_scores, mask)
            
        value_with_attention = torch.matmul(attention_scores, value)

        return value_with_attention, attention_scores

#### Multi Head Attention

In [118]:
class MultiHeadAttention(nn.Module) :
    
    def __init__(self, 
                 n_head: int = 8, 
                 d_model: int = 512, 
                 dropout: float = 0.1, 
                 mask: bool = False,
                 self_attention:bool = True
        ) :

        """
        Args:
            n_head (int): Number of heads. Defaults to 8.
            d_model (int): Dimension of input. Defaults to 512.
            dropout (float): Dropout rate. Defaults to 0.1.
            mask (bool): Whether to mask the attention. Defaults to False.
            self_attention (bool): Whether to use self attention. Defaults to True.
        """

        super(MultiHeadAttention, self).__init__()
        self.n_head = n_head
        self.d_model = d_model
        self.dropout = nn.Dropout(dropout)
        self.self_attention = self_attention

        self.d_k = self.d_v = d_model // n_head
        self.w_qs = nn.Linear(d_model, n_head * self.d_k)
        self.w_ks = nn.Linear(d_model, n_head * self.d_k)
        self.w_vs = nn.Linear(d_model, n_head * self.d_v)

        self.attention = ScaledDotProductAttention(n_head, d_model, mask)

        self.mha_linear = nn.Linear(d_model, d_model)

        nn.init.normal_(self.w_qs.weight, mean = 0, std = np.sqrt(2.0 / (d_model + self.d_k)))
        nn.init.normal_(self.w_ks.weight, mean = 0, std = np.sqrt(2.0 / (d_model + self.d_k)))
        nn.init.normal_(self.w_vs.weight, mean = 0, std = np.sqrt(2.0 / (d_model + self.d_v)))

    def forward(self, x, q = None) :

        """
        Implementation of multi head attention layer.

        Args:
            x (torch.Tensor): Padded input with the shaep batch_len, seq_len, d_model
            q (torch.Tensor): Query with the shape batch_size, seq_len, d_model. Defaults to None.
        
        Returns:
            torch.Tensor: Values with multiheadattention applied. Shape = (batch_size, seq_len, d_model)
        
        Raises:
            ValueError: If mode is cross attention and query passed in forward is None.
            ValueError: If mode is cross attention and shape of query is not same as input coming from encoder.
        
        References:
            https://github.com/jadore801120/attention-is-all-you-need-pytorch/blob/master/transformer/MultiHeadAttention.py
            https://github.com/jadore801120/attention-is-all-you-need-pytorch/blob/master/transformer/Transformer.py
            https://github.com/jadore801120/attention-is-all-you-need-pytorch/blob/master/transformer/PositionalEncoding.py
        """
         
        if not self.self_attention:
            if q is None :
                raise ValueError("q is required for cross attention")
            elif x.size() != q.size() :
                raise ValueError("q and X must have same size")
        else :
            q = x

        key = F.gelu(self.w_ks(x))
        query = F.gelu(self.w_qs(q))
        value = F.gelu(self.w_vs(x))

        ## keeping n_heads as major dimension
        key = key.view(-1, key.size(0), key.size(1), self.d_k)
        query = query.view(-1, query.size(0), query.size(1), self.d_k)
        value = value.view(-1, value.size(0), value.size(1), self.d_v)

        value, attention = self.attention(key, query, value)
        logs(f'values with attention size : {value.size()}')

        value = value.view(value.size(1), value.size(2), -1)
        logs(f'values with attention after reshape size : {value.size()}')

        value = self.dropout(value)

        value = F.gelu(self.mha_linear(value))

        return value, attention

#### Add and layer normalization

In [119]:
class AddLayerNormalization(nn.Module) :

    def __init__(self, sequence_len, d_model) :
        super().__init__()

        self.layer_norm = nn.LayerNorm([sequence_len, d_model])

    def forward(self, x, mha_output) :

        return self.layer_norm(x + mha_output)

#### Point Wise Feedforward

In [120]:
class PointWiseFeedforward(nn.Module) :

    def __init__(self, 
                 d_ff: int = 2048, 
                 d_model: int = 512
    ) -> None :
        
        """
        Args:
            d_ff (int): Intermediate size of the feedforward layer.
            d_model (int):  Size of the embeddings.
        """

        super(PointWiseFeedforward, self).__init__()
        self.linear1 = nn.Linear(d_model, d_ff)
        self.linear2 = nn.Linear(d_ff, d_model)

    def forward(self, x) :

        linear1_output = self.linear1(x)
        linear2_output = self.linear2(F.gelu(linear1_output))

        return linear2_output
        

####  Single Encoder Layer

In [121]:
class EncoderLayer(nn.Module) :

    def __init__(self,
                 n_heads: int = 8,
                 max_seq_len: int = 64,
                 d_model: int = 512,
                 d_ff: int = 2048
        ) -> None :
        
        super(EncoderLayer, self).__init__()
        
        self.mha = MultiHeadAttention(n_heads, d_model, )
        self.layer_norm = AddLayerNormalization(max_seq_len, d_model)
        self.pff = PointWiseFeedforward(d_ff, d_model)
        self.layer_norm2 = AddLayerNormalization(max_seq_len, d_model)

    def forward(self, x) :

        mha_output, mha_attention_scores = self.mha(x)
        norm_output1 = self.layer_norm(x, mha_output)

        pff_output = self.pff(norm_output1)
        norm_output2 = self.layer_norm2(norm_output1, pff_output)

        return norm_output2, mha_attention_scores

## Encoder

In [139]:
class Encoder(nn.Module) :

    def __init__(self,
                 n_layer: int = 6,
                 n_heads: int = 8,
                 max_seq_len: int = 64,
                 d_model: int = 512,
                 d_ff: int = 2048
    ) :
        super(Encoder, self).__init__()
        
        self.encoder = nn.ModuleDict({
            'encoder_layer_{}'.format(i) : 
            (
                EncoderLayer(
                    n_heads,
                    max_seq_len,
                    d_model,
                    d_ff
                )
            ) for i in range(n_layer)
            })

    def forward(self, x) :
        logs(f'input size : {x.size()}')
        for name, layer in self.encoder.items() :
            x, attention_scores = layer(x)
            logs(f'{name} output size : {x.size()}')
        return x

## Experimentation